## ‘What Destroys a Person When that Person Appears to Be Destroying Himself?’

_Fine tunes BERT, RoBERTa, DistilBERT pretrained LMs to classify ITS targets, structures and merges analytic sample linking posts with direct-response comments for fine-tuned LM inference and downstream reflexive thematic analysis. Applies CausalNLP tools to detect post &rarr; comment ruptures in cumulative elicited support conditioned on ITS-labeled expressions and additional post author disclosures._

> its_train_eval_pred.ipynb<br>
> Simone J. Skeen (08-14-2024)

1. [Prepare](#scrollTo=VfO9Lvsq5pOQ)
2. [Preprocess](#scrollTo=st5A4Pp5MFtZ)
3. [Train-Test: BERT, RoBERTa, DistilBERT](#scrollTo=a4f0b99a-a3cc-493a-b74e-0af2973af2eb)
4. [Train-Test-Tune: model x sub/construct-specific](#scrollTo=xxM1ebUSIvA4)
5. [Predict](#scrollTo=6ALab8tymeX6)
6. [Causal inference](#scrollTo=8-MPUSq8J-Vv)

### 1. Prepare
Installs, imports, and downloads requisite models and packages. Organizes RAP-consistent directory structure.
***

**Install**

In [ ]:
%%capture

%pip install causalnlp
%pip install contractions
%pip install unidecode
%pip install wandb

#%pip uninstall -y pyarrow datasets
#%pip install pyarrow datasets

!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.0.0/en_core_web_lg-3.0.0.tar.gz

**Import**

In [ ]:
import contractions
import en_core_web_lg
import gzip
import json
import logging
import numpy as np
import os
import pandas as pd
import re
import sklearn
import spacy
import string
import torch
import wandb.sdk
import warnings

from causalnlp import Autocoder
from causalnlp import Autocoder, CausalInferenceModel
from google.colab import drive
from lightgbm import LGBMClassifier
from sklearn.feature_extraction import text
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import KFold, ParameterGrid, StratifiedKFold, train_test_split
from sklearn.metrics import classification_report, f1_score, matthews_corrcoef, average_precision_score
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.multiclass import type_of_target
from textblob import TextBlob
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification, DistilBertTokenizer, DistilBertForSequenceClassification
from unidecode import unidecode

spacy.cli.download('en_core_web_lg')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.set_option(
              'display.max_columns',
              None,
              )

pd.set_option(
              'display.max_rows',
              None,
              )

warnings.simplefilter(
                      action = 'ignore',
                      category = FutureWarning,
                      )

#!python -m prodigy stats

**Mount gdrive**

In [ ]:
drive.mount(
            '/content/gdrive/',
            force_remount = True,
            )

**Structure directories**

In [ ]:
%cd /content/gdrive/My Drive/Colab/intrapolitical_suicidality
#%cd /content/gdrive/My Drive/#<my_project_folder>

#%mkdir intrapolitical_suicidality

In [ ]:
#%cd intrapolitical_suicidality
#%mkdir inputs outputs code temp

In [ ]:
#%cd inputs
#%mkdir archives data

In [ ]:
#%cd outputs
#%mkdir models tables

In [ ]:
### dir structure TKTK

### 2. Preprocess
Cleans annotated dataset $\mathcal{d}$<sub>annotated</sub>, builds rationale-augmented $\mathcal{d}$<sub>augmented</sub>. Builds and NER-anonymizes analytic sample $\mathcal{D}$<sub>inference</sub>, subsamples $\mathcal{D}$<sub>reflexive</sub>.
***

#### Clean: annotated set $\mathcal{d}$<sub>annotated</sub>

In [ ]:
%%capture
%cd inputs/data

d = pd.read_excel('d_annotated.xlsx')

d.dtypes
d.shape
d.head(3)

**Reformat**

In [ ]:
d.set_index(
            'index',
            inplace = True,
            )

# Encode multinomial target

multi = [
         (d['pb'] == 0) & (d['tb'] == 0),
         (d['pb'] == 1) & (d['tb'] == 0),
         (d['pb'] == 0) & (d['tb'] == 1),
         (d['pb'] == 1) & (d['tb'] == 1),
         ]

label_encode = [
                0,
                1,
                2,
                3,
                ]

d['its'] = np.select(
                     multi,
                     label_encode,
                     )

# Append PB + TB rationales

d['pb_rtnl'] = d['pb_rtnl'].astype(str)
d['tb_rtnl'] = d['tb_rtnl'].astype(str)

d['its_rtnl'] = d['pb_rtnl'] + ' ' + d['tb_rtnl']

# Replace with "." to denote no rationale

rationales = [
              'pb_rtnl',
              'tb_rtnl',
              ]

for rationale in rationales:
    d[rationale] = d[rationale].str.replace(r'0', '.', regex = True)

d['its_rtnl'] = d['its_rtnl'].str.replace(r'0 0', '.', regex = True)

d.dtypes
d.shape
d.head(3)

#### Build: augmented set $\mathcal{d}$<sub>augmented</sub>

In [ ]:
%cd ../../code

In [ ]:
%%writefile augment.py

import pandas as pd

def augment_training_data_with_rationales(row):
    """
    Identifies all pos(1) ITS rows, duplicates as new row below, replaces new row 'text' cell with appended PB-TB 'rationale' cell
    """
    if row['its'] > 0:
        new_row = row.copy()
        new_row['text'] = row['its_rtnl']
        return pd.concat([row, new_row], axis = 1)
    else:
        return row


In [ ]:
from augment import augment_training_data_with_rationales

#import pandas as pd

d = d.apply(
            augment_training_data_with_rationales,
            axis = 1,
            )

d_aug = pd.concat(
                  d.tolist(),
                  axis = 1,
                  ).T

# Reset index

d_aug.reset_index(
                  drop = True,
                  inplace = True,
                  )

# Verify...

d_aug.head(3)
d_aug.shape
d_aug['its'].value_counts()

**Clean**

In [ ]:
# Clear empty cells

d_aug = d_aug[d_aug.text != ' ']

# Clear '0' cells

d_aug = d_aug[d_aug['text'] != '0']

# Clear empty + '0' by visually confirmed p_id (failsafe)

# Define the list of p_id values to drop
del_id = [
          '7m1105',
          'fsncni',
          '10rv1kt',
          'le5egj',
          'onpzzm',
          'xkw7wi',
          'pdearh',
          'hkqtcw',
          'bq8omz',
          'f9mf2g',
          'gf4ukm',
          '9jpshz',
          'x7l9ck',
          '98oo27',
          '373hrz',
          '38au87',
          '9wxbvv',
          'iislhk',
          'ypa71a',
          'zd31tr',
          '2pulw8',
          '1yjsba',
          'ukw36h',
          '2699w9',
          '8q029b',
          'skgzms',
         ]

# Drop rows with specified p_id values

d_aug = d_aug[~d_aug['p_id'].isin(del_id)]

# Excise deprecated '[REDACTED]' NER tags

d_aug['text'] = d_aug['text'].str.replace(r'\[REDACTED\]', ' ', regex = True)

# Confirm...

#rdct_rows = d_aug[d_aug['text'].str.contains(r'\[REDACTED\]', regex = True)]
#print(rdct_rows)

# Excise encoding artifacts

artifacts = [
             '0',
             'Ã,',
             'Â…Ã',
             'Â',
             '€',
             '¢',
             'ƒ',
             '¬',
             '£',
             ]

for artifact in artifacts:
    d_aug['text'] = d_aug['text'].str.replace(
                                              artifact,
                                              ' ',
                                              regex = True,
                                              )

d_aug.shape
d_aug.head(3)

In [ ]:
%%writefile -a augment.py

import pandas as pd

def dummy_code_augmented_rows(df):
    """
    Identifies all rationale-augmented rows in df, dummy codes for deletion prior to evaluation.
    """

    df = df.reset_index(drop = True)

    df['aug'] = 0

    for i in range(1, len(df)):
        if df.at[i, 'its_rtnl'] != '.' and df.at[i, 'its_rtnl'] == df.at[i-1, 'its_rtnl']:
            df.at[i, 'aug'] = 1

    return df

In [ ]:
from augment import dummy_code_augmented_rows

d_aug = dummy_code_augmented_rows(d_aug)

d_aug[[
       'aug',
       'hate',
       'libl',
       'lone',
       'recp',
       'pb',
       'tb',
       ]].apply(pd.Series.value_counts)

d_aug.head(3)

In [ ]:
%cd ../inputs/data

d_aug.to_excel('d_augmented.xlsx')

#### Build: prediction set $\mathcal{D}$<sub>inference</sub>

In [ ]:
archives_path = '/content/gdrive/MyDrive/Colab/intrapolitical_suicidality/inputs/archives/'

dir_p = f'{archives_path}sw_complete-posts.json.gz'

dir_c = f'{archives_path}sw_complete-comments.json.gz'


# d_p = posts

d_p = pd.DataFrame(json.loads(l) for l in gzip.open(
                                                    dir_p,
                                                    'rb',
                                                    )
                   )

d_p = d_p.drop_duplicates(subset = 'id')

d_p['date'] = pd.to_datetime(
                             d_p.created_utc,
                             unit = 's',
                             )

d_p.set_index(
              'date',
              drop = False,
              inplace = True,
              )

d_p = d_p.loc[(d_p['date'] >= '2018-01-01') & (d_p['date'] < '2023-01-01')]

d_p = d_p[~d_p['selftext'].isin(['[deleted]', '[removed]'])]


# d_c = comments

d_c = pd.DataFrame(json.loads(l) for l in gzip.open(
                                                    dir_c,
                                                    'rb',
                                                    )
                   )

d_c = d_c.drop_duplicates(subset = 'id')

d_c['date'] = pd.to_datetime(
                             d_c.created_utc,
                             unit = 's',
                             )

d_c.set_index(
              'date',
              drop = False,
              inplace = True,
              )

d_c = d_c.loc[(d_c['date'] >= '2018-01-01') & (d_c['date'] < '2023-01-01')]

d_c = d_c[~d_c['body'].isin(['[deleted]', '[removed]'])]


d_p.shape
d_c.shape

**Delete comment-level prefix**

In [ ]:
d_c['link_id'] = d_c['link_id'].str.replace('t3_', '')

**Disambiguate 'date', 'id'**

In [ ]:
d_c.rename(
           columns = {'date': 'c_date'},
           inplace = True,
           )
d_c.rename(
           columns = {'id': 'c_id'},
           inplace = True,
           )

**Harmonize 1:_n_ merge variable**

In [ ]:
d_c.rename(
           columns = {'link_id': 'id'},
           inplace = True,
           )
#d_c.columns

In [ ]:
d = d_p.merge(
              d_c,
              how = 'left',
              on = 'id',
              suffixes = ('_post', '_comment')
              )

d.shape
#d.columns
#d.head(30)

In [ ]:
# Housekeeping

d = d[[
       'author_post',
       'date',
       'id',
       'num_comments',
       'selftext',
       'subreddit_post',
       'title',
       'author_comment',
       'subreddit_comment',
       'body',
       ]].copy()

d.rename(
         columns = {
                    'author_post' : 'p_au',
                    'date' : 'p_date',
                    'num_comments' : 'n_cmnt',
                    'selftext' : 'text',
                    'subreddit_post' : 'p_sbrt',
                    'title' : 'p_titl',
                    'comment_date' : 'c_date',
                    'author_comment' : 'c_au',
                    'subreddit_comment' : 'c_sbrt',
                    'body' : 'c_text',
                    }, inplace = True,
           )


# Pseudo-word token for repeat rows

d['dupl'] = d.duplicated(
                        subset = 'text',
                        keep = 'first',
                        )

d.loc[d['dupl'], 'text'] = '<|RPT|>'
d = d.drop(columns = ['dupl'])

d['dupl'] = d.duplicated(
                         subset = 'p_titl',
                         keep = 'first',
                         )

d.loc[d['dupl'], 'p_titl'] = '<|RPT|>'
d = d.drop(columns = ['dupl'])


# Post-comment block var

# New block
d['new_block'] = d['text'] != '<|RPT|>'

# _Sum_ blocks for blockid
d['block'] = (d['new_block'].cumsum() + 1).where(d['new_block'], 0).astype(int)

# Forward fill
d['block'] = d['block'].replace(0, method = 'ffill').astype(int)

        ### SJS 12/30: preserve new_block var for sense-check (for now)

# Drop 'new_block'
#post_comments = post_comments.drop(columns=['new_block'])

# Dummy for p_author replying to comments

d['p_au_reply'] = np.where(d['p_au'] == d['c_au'], 1, 0)

d.shape
d.head(20)

**Subsample and export: $\mathcal{D}$<sub>reflexive</sub>**

In [ ]:
%pwd

In [ ]:
# Subsample

sampled_blocks = d['block'].sample(
                                  n = 1000,
                                  replace = False,
                                  random_state = 56,
                                  )

# Filter

d_reflexive = d[d['block'].isin(sampled_blocks)]
d_reflexive.shape
#d_reflexive.head(30)

d_reflexive.to_excel('d_reflexive.xlsx')

**Restrict (2021-2022) and export: $\mathcal{D}$<sub>inference</sub>**

In [ ]:
d_inference = d.loc[(d['p_date'] >= '2021-01-01') & (d['p_date'] < '2023-01-01')]

d_inference.head(3)
d_inference.tail(3)
d_inference.shape

d_inference.to_csv('d_inference.csv')

### 3. Train-Test: BERT, RoBERTa, DistilBERT
Trains baseline BERT, RoBERTa, and DistilBERT, using rationale-augmented data $\mathcal{d}$<sub>augmented</sub>, iterating over intrapolitical theory targets. <br>Evaluates using de-augmented data. Outputs model x target _F_<sub>1</sub> (macro) performance scores.
***

In [ ]:
d_augmented = pd.read_excel('d_augmented.xlsx')

d_augmented = d_augmented[[
                           'text',
                           'aug',
                           'hate',
                           'libl',
                           'lone',
                           'recp',
                           'pb',
                           'tb',
                           ]].copy()

d_augmented[[
             'aug',
             'hate',
             'libl',
             'lone',
             'recp',
             'pb',
             'tb',
             ]].apply(pd.Series.value_counts)

d_augmented.shape
d_augmented.head(3)

In [ ]:
%cd ../../outputs/tables

In [ ]:
#%%capture

targets_and_class_weights = {
                             'hate': [
                                      0.6416, ### neg_0 class_weight
                                      2.2646, ### pos_1 class_weight
                                      ],
                             'libl': [
                                      0.5207,
                                      12.5688,
                                      ],
                             'lone': [
                                      0.6911,
                                      1.8084,
                                      ],
                             'recp': [
                                      0.5314,
                                      8.4496,
                                      ],
                               'pb': [
                                      0.6735,
                                      1.9411,
                                      ],
                               'tb': [
                                      0.7281,
                                      1.5960,
                                      ],
                              }


models = {
          'bert': (
                   BertForSequenceClassification,
                   BertTokenizer,
                   'bert-base-uncased',
                   ),

          'roberta': (
                      RobertaForSequenceClassification,
                      RobertaTokenizer,
                      'roberta-base',
                      ),

          'distilbert': (
                         DistilBertForSequenceClassification,
                         DistilBertTokenizer,
                         'distilbert-base-uncased',
                         )
          }


k_fold = StratifiedKFold(
                         n_splits = 5,
                         shuffle = True,
                         random_state = 56,
                         )

results = []

# Check CUDA

print("CUDA: ", torch.cuda.is_available())
use_cuda = torch.cuda.is_available()

# Best F_1 tracking

best_f1_scores = {target: {'score': 0, 'model': None} for target in targets_and_class_weights}

# Training loop: target x model

for target, class_weights in targets_and_class_weights.items():

    print("\n======================================================================================")
    print(f"Label: {target}")
    print("======================================================================================")

    for model_name, (
                     model_class,
                     tokenizer_class,
                     pretrained_model_name,
                     ) in models.items():

        print(f"\nFine-tuning {model_name} for {target}")
        print("--------------------------------------------------------------------------------------")

        fold_results = {
                        'target': target,
                        'model': model_name,
                        }
        metrics = {
                   'f1_macro': [],
                   'mcc': [],
                   'auprc': [],
                   }


        # Extract text, targets

        X = d_augmented['text'].values
        y = d_augmented[target].values

        # Determine target type

        target_type = 'binary' if len(np.unique(y)) <= 2 else 'multiclass'

        # Convert target (as needed)

        if target_type == 'binary':
            le = LabelEncoder()
            y = le.fit_transform(y)

        # Iterate over 5-fold cv

        for fold_idx, (train_idx, valid_idx) in enumerate(k_fold.split(X, y)):
            X_train, y_train = X[train_idx], y[train_idx]
            X_valid, y_valid = X[valid_idx], y[valid_idx]

            # Parse by 'aug'

            train_aug_mask = d_augmented.iloc[train_idx]['aug'] == 1
            valid_no_aug_mask = d_augmented.iloc[valid_idx]['aug'] != 1

            X_train_aug = X_train[train_aug_mask]
            y_train_aug = y_train[train_aug_mask]
            X_valid_no_aug = X_valid[valid_no_aug_mask]
            y_valid_no_aug = y_valid[valid_no_aug_mask]

            # Tokenize

            tokenizer = tokenizer_class.from_pretrained(pretrained_model_name)

            encoded_train_aug = tokenizer(
                                          X_train_aug.tolist(),
                                          padding = True,
                                          truncation = True,
                                          return_tensors = 'pt',
                                          )

            encoded_valid_no_aug = tokenizer(
                                             X_valid_no_aug.tolist(),
                                             padding = True,
                                             truncation = True,
                                             return_tensors = 'pt',
                                             )

            train_dataset_aug = TensorDataset(
                                              encoded_train_aug['input_ids'],
                                              encoded_train_aug['attention_mask'],
                                              torch.tensor(y_train_aug)
                                              )

            valid_dataset_no_aug = TensorDataset(
                                                 encoded_valid_no_aug['input_ids'],
                                                 encoded_valid_no_aug['attention_mask'],
                                                 torch.tensor(y_valid_no_aug)
                                                 )

            train_loader_aug = DataLoader(
                                          train_dataset_aug,
                                          batch_size = 4,
                                          shuffle = True,
                                          )

            valid_loader_no_aug = DataLoader(
                                             valid_dataset_no_aug,
                                             batch_size = 4,
                                             shuffle = False,
                                             )

            # Instantiate model

            model = model_class.from_pretrained(pretrained_model_name)

            # Migrate to CUDA (if available)

            if use_cuda:
                model = model.cuda()

            # Fine-tune model on training data with 'aug' rows

            optimizer = torch.optim.AdamW(
                                          model.parameters(),
                                          lr = 2e-5,
                                          )

            # Training with 'aug'

            model.train()

            # Define the loss criterion with class weights

            criterion = CrossEntropyLoss(
                                         weight = torch.tensor(
                                                               class_weights,
                                                               dtype = torch.float,
                                                               ).cuda() if use_cuda else torch.tensor(
                                                                                                      class_weights,
                                                                                                      dtype = torch.float,
                                                                                                      )
                                         )

            for epoch in range(2):  ### range(n) = n epochs
                for i, batch in enumerate(train_loader_aug):
                    input_ids, attention_mask, labels = batch
                    labels = labels.long()
                    if use_cuda:
                        input_ids, attention_mask, labels = input_ids.cuda(), attention_mask.cuda(), labels.cuda()
                    optimizer.zero_grad()
                    outputs = model(
                                    input_ids,
                                    attention_mask = attention_mask,
                                    )
                    logits = outputs.logits
                    loss = criterion(
                                     logits,
                                     labels,
                                     )
                    loss.backward()
                    optimizer.step()

            # Evaluate model on validation data without 'aug'

            model.eval()
            all_predictions = []
            all_true_labels = []
            with torch.no_grad():
                for batch in valid_loader_no_aug:
                    input_ids, attention_mask, labels = batch
                    if use_cuda:  # Added this line
                        input_ids, attention_mask, labels = input_ids.cuda(), attention_mask.cuda(), labels.cuda()  # Added this line
                    outputs = model(
                                    input_ids,
                                    attention_mask = attention_mask,
                                    )
                    logits = outputs.logits
                    predictions = torch.argmax(
                                               logits,
                                               dim = 1,
                                               ).tolist()
                    all_predictions.extend(predictions)
                    all_true_labels.extend(labels.tolist())

            # Calculate evaluation metrics without 'aug'

            f1_macro = f1_score(
                                all_true_labels,
                                all_predictions,
                                average = 'macro',
                                )

            mcc = matthews_corrcoef(
                                    all_true_labels,
                                    all_predictions,
                                    )

            auprc = average_precision_score(
                                            all_true_labels,
                                            all_predictions,
                                            average='macro',
                                            )

            metrics['f1_macro'].append(f1_macro)
            metrics['mcc'].append(mcc)
            metrics['auprc'].append(auprc)

        # Compute average metrics for the fold

        fold_results.update({metric: np.mean(values) for metric, values in metrics.items()})
        fold_results.update({metric + '_std': np.std(values) for metric, values in metrics.items()})

        # Append fold results to overall results

        results.append(fold_results)

        # Update best F_1 score

        if np.mean(metrics['f1_macro']) > best_f1_scores[target]['score']:
            best_f1_scores[target]['score'] = np.mean(metrics['f1_macro'])
            best_f1_scores[target]['model'] = model_name

# Display and export results

print("\n--------------------------------------------------------------------------------------")
print(f"Summary")
print("--------------------------------------------------------------------------------------")

for target, info in best_f1_scores.items():
    print(f"Best F1 (macro) for {target}: {info['score']} achieved by {info['model']}")

d_perf = pd.DataFrame(results)
#print(d_perf)
d_perf.head(24)

d_perf.to_excel('d_bl_performance.xlsx')

### 4. Train-Test-Tune: model x sub/construct-specific
Builds stratified train-test sets, searches hyperparam space to optimize highest-performing sub/construct x pretrained model configs.
***

In [ ]:
%cd ../../code

#### Build train-test sets: $\mathcal{d}$<sub>train</sub> (rationale-augmented), $\mathcal{d}$<sub>test</sub>

**_stratified_train_test_split_with_rationales_**

In [ ]:
%%writefile train.py

import pandas as pd
from sklearn.model_selection import train_test_split

def stratified_train_test_split_with_rationales(df, test_size = 0.2, random_state = 56):
    """
    Splits df into target-stratified train and test sets, partitions 'rationales (aug = 1) to train set
    """

    aug_rows = df[df['aug'] == 1]
    non_aug_rows = df[df['aug'] != 1]

    train_non_aug, test_non_aug = train_test_split(
                                                   non_aug_rows,
                                                   test_size = test_size,
                                                   stratify = non_aug_rows['targets'],
                                                   random_state = random_state,
                                                   )

    d_train = pd.concat([
                         train_non_aug,
                         aug_rows,
                         ])

    print("--------------------------------------------------------------------------------------")
    print(f"d_train: Augmented training data")
    print("--------------------------------------------------------------------------------------")

    d_train = d_train.sample(
                             frac = 1,
                             random_state = random_state,
                             ).reset_index(drop = True)

    print(d_train.shape)
    print(d_train['aug'].value_counts(normalize = True))
    print(d_train['targets'].value_counts(normalize = True))
    print(d_train.head(6))

    d_test = test_non_aug
    print("\n--------------------------------------------------------------------------------------")
    print(f"d_test: De-augmented evaluation data")
    print("--------------------------------------------------------------------------------------")
    print(d_test.shape)
    print(d_test['aug'].value_counts(normalize = True))
    print(d_test['targets'].value_counts(normalize = True))
    print(d_test.head(6))

    print("\n--------------------------------------------------------------------------------------")
    print("Confirming 'aug' deletion")
    print("--------------------------------------------------------------------------------------")
    d_train = d_train.drop('aug', axis = 1)
    print(d_train.shape)
    print(d_train.head(1))

    d_test = d_test.drop('aug', axis = 1)
    print(d_test.shape)
    print(d_test.head(1))

    return d_train, d_test

#### Self-hatred: 'hate'

In [ ]:
# Streamline

d = d_augmented[[
                 'text',
                 'hate',
                 'aug',
                 ]].copy()

d.columns = [
             'text',
             'targets',
             'aug',
              ]

d.dtypes
d.head(3)

In [ ]:
from train import stratified_train_test_split_with_rationales

d_train, d_test = stratified_train_test_split_with_rationales(d)

In [ ]:
%cd ../temp

d_train.to_excel('d_train.xlsx')
d_test.to_excel('d_test.xlsx')

**Training, hyperparam tuning**

In [ ]:
# Define tokenizer, model class, pretrained model

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model_class = DistilBertForSequenceClassification
pretrained_model_name = 'distilbert-base-uncased'

# Define label

target = 'hate'

# Define class weights

class_weights = torch.tensor(
                             [
                              0.6416,  # neg_0
                              2.2646,  # pos_1
                              ], dtype=torch.float,
                              )

# Define save_path

%cd ../outputs/models
%mkdir hate_distilbert

save_path = '/content/gdrive/My Drive/Colab/intrapolitical_suicidality/outputs/models/hate_distilbert'

In [ ]:
#%cd ../../code

**_tune_and_optimize_model_hyperparams_**

In [ ]:
%%writefile -a train.py

import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import CrossEntropyLoss
from transformers import BertForSequenceClassification, AdamW
from sklearn.metrics import f1_score
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

# Define Fx

def tune_and_optimize_model_hyperparams(tokenizer, model_class, pretrained_model_name, d_train, d_test, target, class_weights, save_path):
    """
    Tune and optimize model hyperparameters.

    Parameters:
    tokenizer: Pre-trained tokenizer.
    model_class: Pre-trained model class.
    pretrained_model_name: Name of the pre-trained model.
    d_train: Training dataset.
    d_test: Test dataset.
    target: Target variable for classification.
    class_weights: Weights for each class.
    save_path: Path to save the best model.

    Returns:
    d_test: Test dataset with predictions and probabilities.
    """

    # Check CUDA

    use_cuda = torch.cuda.is_available()
    print("CUDA: ", use_cuda)

    print("======================================================================================")
    print(f"Optimizing: {pretrained_model_name}\nLabel: {target}")
    print("======================================================================================")

    # Define hyperparam grid

    param_grid = {
        'per_gpu_batch_size': [8, 16],
        'weight_decay': [0, 0.3],
        'learning_rate': [2e-5, 3e-5, 5e-5],
        'warmup_steps': [0, 500],
        'num_epochs': [2, 3]
    }

    # Class weights

    if use_cuda:
        class_weights = class_weights.cuda()

    # Tokenize

    encoded_train = tokenizer(
        d_train['text'].tolist(),
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    encoded_test = tokenizer(
        d_test['text'].tolist(),
        padding=True,
        truncation=True,
        return_tensors='pt'
    )

    train_labels = torch.tensor(d_train['targets'].values)
    test_labels = torch.tensor(d_test['targets'].values)

    train_dataset = TensorDataset(
        encoded_train['input_ids'],
        encoded_train['attention_mask'],
        train_labels
    )
    test_dataset = TensorDataset(
        encoded_test['input_ids'],
        encoded_test['attention_mask'],
        test_labels
    )

    # Initialize tracking variables

    best_f1_macro = 0
    best_params = None
    best_model_state = None
    best_predictions = []
    best_probabilities = []

    f1_scores = []

    # Gradient accumulation steps

    accumulation_steps = 2

    # Grid search

    for params in ParameterGrid(param_grid):
        train_loader = DataLoader(
            train_dataset,
            batch_size=params['per_gpu_batch_size'],
            shuffle=True
        )
        test_loader = DataLoader(
            test_dataset,
            batch_size=params['per_gpu_batch_size'],
            shuffle=False
        )

        print(f"\nTotal training rows: {len(train_dataset)}")
        print(f"Total evaluation rows: {len(test_dataset)}")
        print(f"Training batch size: {params['per_gpu_batch_size']}")
        print(f"Evaluation batch size: {params['per_gpu_batch_size']}")
        print(f"Total training batches: {len(train_loader)}")
        print(f"Total evaluation batches: {len(test_loader)}")
        print("\n")

        model = model_class.from_pretrained(pretrained_model_name)
        if use_cuda:
            model.cuda()

        optimizer = AdamW(
            model.parameters(),
            lr=params['learning_rate'],
            weight_decay=params['weight_decay']
        )

        criterion = CrossEntropyLoss(weight=class_weights)

        # Training loop

        for epoch in range(params['num_epochs']):
            model.train()
            optimizer.zero_grad()
            for i, batch in enumerate(tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{params['num_epochs']}", leave=True)):
                input_ids, attention_mask, labels = batch
                if use_cuda:
                    input_ids, attention_mask, labels = input_ids.cuda(), attention_mask.cuda(), labels.cuda()
                outputs = model(
                    input_ids,
                    attention_mask=attention_mask
                )
                loss = criterion(outputs.logits, labels)
                loss = loss / accumulation_steps
                loss.backward()
                if (i + 1) % accumulation_steps == 0:
                    optimizer.step()
                    optimizer.zero_grad()

        # Eval loop

        model.eval()
        all_predictions = []
        all_true_labels = []
        all_probabilities = []
        with torch.no_grad():
            progress_bar = tqdm(total=len(test_loader), desc="Evaluating", leave=True)
            for batch in test_loader:
                input_ids, attention_mask, labels = batch
                if use_cuda:
                    input_ids, attention_mask, labels = input_ids.cuda(), attention_mask.cuda(), labels.cuda()
                outputs = model(
                    input_ids,
                    attention_mask=attention_mask
                )
                probabilities = torch.softmax(outputs.logits, dim=1)
                predictions = torch.argmax(probabilities, dim=1).cpu().tolist()
                all_predictions.extend(predictions)
                all_true_labels.extend(labels.cpu().tolist())
                all_probabilities.extend(probabilities.cpu().tolist())
                progress_bar.update(1)
            progress_bar.close()

        current_f1_macro = f1_score(all_true_labels, all_predictions, average='macro')
        f1_scores.append(current_f1_macro)
        print(f"\nCurrent F1 macro for params {params}: {current_f1_macro}")

        if current_f1_macro > best_f1_macro:
            best_f1_macro = current_f1_macro
            best_params = params
            best_model_state = model.state_dict()
            best_predictions = all_predictions
            best_probabilities = all_probabilities

    if len(best_predictions) == len(d_test):
        d_test['predicted_labels'] = best_predictions
        d_test['predicted_probabilities'] = best_probabilities
    else:
        print("Error: Length of predictions does not match length of test set")

    # Output test df with predictions and raw probabilities

    print("--------------------------------------------------------------------------------------")
    print(f"Summary: {target}")
    print("--------------------------------------------------------------------------------------")

    print(d_test.head(6))
    d_test.to_excel('d_test_preds.xlsx')

    if best_model_state:
        model_path = f"{save_path}/{target}_best_model.bin"
        torch.save(best_model_state, model_path)
        print("\nBest model saved with F1 macro:", best_f1_macro)
        print("Best hyperparameters:", best_params)

    # Print M (SD) of F_1 scores

    f1_mean = sum(f1_scores) / len(f1_scores)
    f1_std = (sum((x - f1_mean) ** 2 for x in f1_scores) / len(f1_scores)) ** 0.5
    print(f"Mean F1 macro: {f1_mean}")
    print(f"Standard deviation of F1 macro: {f1_std}")

    return d_test

In [ ]:
%cd ../../code

from train import tune_and_optimize_model_hyperparams

d_test = tune_and_optimize_model_hyperparams(tokenizer, model_class, pretrained_model_name, d_train, d_test, target, class_weights, save_path)

#### Loneliness: 'lone'

In [ ]:
# Streamline

d = d_augmented[[
                 'text',
                 'lone',
                 'aug',
                 ]].copy()

d.columns = [
             'text',
             'targets',
             'aug',
              ]

d.dtypes
d.head(3)

In [ ]:
#from train import stratified_train_test_split_with_rationales

d_train, d_test = stratified_train_test_split_with_rationales(d)

In [ ]:
%cd ../temp

d_train.to_excel('d_train.xlsx')
d_test.to_excel('d_test.xlsx')

In [ ]:
# Define tokenizer, model class, pretrained model

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model_class = DistilBertForSequenceClassification
pretrained_model_name = 'distilbert-base-uncased'

# Define label

target = 'lone'

# Define class weights

class_weights = torch.tensor(
                             [
                              0.6911,  # neg_0
                              1.8084,  # pos_1
                              ], dtype = torch.float,
                             )

# Define save_path

%cd ../outputs/models
%mkdir lone_distilbert

save_path = '/content/gdrive/My Drive/Colab/intrapolitical_suicidality/outputs/models/lone_distilbert'

In [ ]:
%cd lone_distilbert

d_test = tune_and_optimize_model_hyperparams(tokenizer, model_class, pretrained_model_name, d_train, d_test, target, class_weights, save_path)

#### Thwarted belongingness: 'tb'

In [ ]:
# Streamline

d = d_augmented[[
                 'text',
                 'tb',
                 'aug',
                 ]].copy()

d.columns = [
             'text',
             'targets',
             'aug',
              ]

d.dtypes
d.head(3)

In [ ]:
d_train, d_test = stratified_train_test_split_with_rationales(d)

In [ ]:
%cd /content/gdrive/My Drive/Colab/intrapolitical_suicidality/outputs/models

In [ ]:
# Define tokenizer, model class, pretrained model

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_class = BertForSequenceClassification
pretrained_model_name = 'bert-base-uncased'

# Define label

target = 'tb'

# Define class weights

class_weights = torch.tensor(
                             [
                              0.7281,  # neg_0
                              1.5960,  # pos_1
                              ], dtype = torch.float,
                             )

# Define save_path

#%cd ../outputs/models
#%mkdir tb_bert

save_path = '/content/gdrive/My Drive/Colab/intrapolitical_suicidality/outputs/models/tb_bert'

In [ ]:
%cd tb_bert

d_test = tune_and_optimize_model_hyperparams(tokenizer, model_class, pretrained_model_name, d_train, d_test, target, class_weights, save_path)

#### Perceived burdensomeness: 'pb'

In [ ]:
# Streamline

d = d_augmented[[
                 'text',
                 'pb',
                 'aug',
                 ]].copy()

d.columns = [
             'text',
             'targets',
             'aug',
              ]

d.dtypes
d.head(3)

In [ ]:
from train import stratified_train_test_split_with_rationales

d_train, d_test = stratified_train_test_split_with_rationales(d)

In [ ]:
# Define tokenizer, model class, pretrained model

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_class = BertForSequenceClassification
pretrained_model_name = 'bert-base-uncased'

# Define label

target = 'pb'

# Define class weights

class_weights = torch.tensor(
                             [
                              0.6735,  # neg_0
                              1.9411,  # pos_1
                              ], dtype = torch.float,
                             )

# Define save_path

%cd ../outputs/models
%mkdir pb_bert

save_path = '/content/gdrive/My Drive/Colab/intrapolitical_suicidality/outputs/models/pb_bert'

In [ ]:
%cd pb_bert

d_test = tune_and_optimize_model_hyperparams(tokenizer, model_class, pretrained_model_name, d_train, d_test, target, class_weights, save_path)

### 5. Predict
Defines functions to tokenize and label $\mathcal{D}$<sub>inference</sub>, $\mathcal{D}$<sub>reflexive</sub> posts with classification models. Predicts labels.
***

**_ner_redact_post_texts_**

In [ ]:
%cd ../../code

In [ ]:
%%writefile predict.py

import spacy
nlp = spacy.load('en_core_web_lg')

def ner_redact_post_texts(p_text):
    """
    Redacts all named entities recognized by spaCy EntityRecognizer, replaces with <|PII|> pseudo-word token.
    """
    ne = list(
              [
               'PERSON',   ### people, including fictional
               'NORP',     ### nationalities or religious or political groups
               'FAC',      ### buildings, airports, highways, bridges, etc.
               'ORG',      ### companies, agencies, institutions, etc.
               'GPE',     ### countries, cities, states
               'LOC',      ### non-GPE locations, mountain ranges, bodies of water
               'PRODUCT',  ### objects, vehicles, foods, etc. (not services)
               'EVENT',    ### named hurricanes, battles, wars, sports events, etc.
               ]
                )

    doc = nlp(p_text)
    ne_to_remove = []
    final_string = str(p_text)
    for sent in doc.ents:
        if sent.label_ in ne:
            ne_to_remove.append(str(sent.text))
    for n in range(len(ne_to_remove)):
        final_string = final_string.replace(
                                            ne_to_remove[n],
                                            '<|PII|>',
                                            )
    return final_string

**_load_model_**

In [ ]:
%%writefile -a predict.py

import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification
from tqdm import tqdm
import pandas as pd

def load_model(model_path, model_class, pretrained_model_name):
    """
    Loads a pre-trained fine-tined LM from a specified path.
    """
    model = model_class.from_pretrained(pretrained_model_name)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

**_preprocess_data_**

In [ ]:
%%writefile -a predict.py

import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification
from tqdm import tqdm
import pandas as pd

def preprocess_data(tokenizer, texts):
    """
    Tokenizes a list of texts using the specified LM-specific tokenizer.
    """
    encoded_texts = tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    return encoded_texts

**_predict_**

In [ ]:
%%writefile -a predict.py

import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification
from tqdm import tqdm
import pandas as pd

def predict(model, tokenizer, texts, batch_size = 8, use_cuda = True):
    """
    Predicts labels and probabilities for a list of texts using the specified model and tokenizer.
    """
    print(f"Total number of texts to predict: {len(texts)}")
    encoded_texts = preprocess_data(tokenizer, texts)
    dataset = TensorDataset(encoded_texts['input_ids'], encoded_texts['attention_mask'])
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    print(f"Batch size: {batch_size}")
    print(f"Total number of batches: {len(data_loader)}")

    if use_cuda:
        model.cuda()

    all_predictions = []
    all_probabilities = []

    with torch.no_grad():
        progress_bar = tqdm(total=len(data_loader), desc="Predicting", leave=False)
        for batch in data_loader:
            input_ids, attention_mask = batch
            if use_cuda:
                input_ids, attention_mask = input_ids.cuda(), attention_mask.cuda()

            outputs = model(input_ids, attention_mask=attention_mask)
            probabilities = torch.softmax(outputs.logits, dim=1)
            predictions = torch.argmax(probabilities, dim=1).cpu().tolist()
            all_predictions.extend(predictions)
            all_probabilities.extend(probabilities.cpu().tolist())
            progress_bar.update(1)
        progress_bar.close()

    return all_predictions, all_probabilities

In [ ]:
%cd ../inputs/data

#d_reflexive = pd.read_excel('d_reflexive.xlsx')

#d_reflexive.shape
#d_reflexive.dtypes
#d_reflexive.head(3)

d_inference = pd.read_csv('d_inference.csv')

d_inference.shape
d_inference.dtypes
d_inference.head(3)

In [ ]:
%cd ../../code

from predict import ner_redact_post_texts, load_model, preprocess_data, predict

d_inference = d_inference.drop('Unnamed: 0', axis = 1)
d_inference = d_inference.reset_index(drop = True)

d_inference['text'] = d_inference['text'].astype(str)

d_inference.head(3)

In [ ]:
d_reflexive = d_reflexive.drop('Unnamed: 0', axis = 1)
d_reflexive = d_reflexive.reset_index(drop = True)

d_reflexive['text'] = d_reflexive['text'].astype(str).apply(lambda i: ner_redact_post_texts(i))

d_reflexive.head(3)

    ### SJS 8/9: move all this housekeeping shit into the Preprocess section...

**'hate'**

In [ ]:
%pwd

In [ ]:
#from predict import load_model, predict

models_path = '/content/gdrive/My Drive/Colab/intrapolitical_suicidality/outputs/models/'

pretrained_model_name = 'distilbert-base-uncased'
model_class = DistilBertForSequenceClassification
tokenizer_class = DistilBertTokenizer
model_path = f'{models_path}hate_distilbert/hate_best_model.bin'

target = 'hate'

tokenizer = tokenizer_class.from_pretrained(pretrained_model_name)

model = load_model(
                   model_path,
                   model_class,
                   pretrained_model_name,
                   )

#texts = d_reflexive['text'].tolist()
texts = d_inference['text'].tolist()
#d_inference['text'] = d_inference['text'].astype(str)

predictions, probabilities = predict(
                                     model,
                                     tokenizer,
                                     texts,
                                     )

#d_reflexive[f'{target}_label'] = predictions
#d_reflexive[f'{target}_proba'] = probabilities
d_inference[f'{target}_label'] = predictions
d_inference[f'{target}_proba'] = probabilities

#d_reflexive.head(3)
d_inference.head(3)

**'lone'**

In [ ]:
%cd /content/gdrive/My Drive/Colab/intrapolitical_suicidality/inputs/data

pretrained_model_name = 'distilbert-base-uncased'
model_class = DistilBertForSequenceClassification
tokenizer_class = DistilBertTokenizer
model_path = f'{models_path}lone_distilbert/lone_best_model.bin'

target = 'lone'

tokenizer = tokenizer_class.from_pretrained(pretrained_model_name)

model = load_model(
                   model_path,
                   model_class,
                   pretrained_model_name,
                   )

#texts = d_reflexive['text'].tolist()
texts = d_inference['text'].tolist()

predictions, probabilities = predict(
                                     model,
                                     tokenizer,
                                     texts,
                                     )

#d_reflexive[f'{target}_label'] = predictions
#d_reflexive[f'{target}_proba'] = probabilities
d_inference[f'{target}_label'] = predictions
d_inference[f'{target}_proba'] = probabilities

#d_reflexive.head(3)
d_inference.head(3)

d_inference.to_csv('d_inference_hate_lone.csv')

**'tb'**

In [ ]:
%cd /content/gdrive/My Drive/Colab/intrapolitical_suicidality/inputs/data

pretrained_model_name = 'bert-base-uncased'
model_class = BertForSequenceClassification
tokenizer_class = BertTokenizer
model_path = f'{models_path}tb_bert/tb_best_model.bin'

target = 'tb'

tokenizer = tokenizer_class.from_pretrained(pretrained_model_name)

model = load_model(
                   model_path,
                   model_class,
                   pretrained_model_name,
                   )

#texts = d_reflexive['text'].tolist()
texts = d_inference['text'].tolist()

predictions, probabilities = predict(
                                     model,
                                     tokenizer,
                                     texts,
                                     )

#d_reflexive[f'{target}_label'] = predictions
#d_reflexive[f'{target}_proba'] = probabilities
d_inference[f'{target}_label'] = predictions
d_inference[f'{target}_proba'] = probabilities

#d_reflexive.head(3)
d_inference.head(3)

d_inference.to_csv('d_inference_hate_lone_tb.csv')

**'pb'**

In [ ]:
%cd /content/gdrive/My Drive/Colab/intrapolitical_suicidality/inputs/data

d_inference = pd.read_csv('d_inference_hate_lone_tb.csv')
d_inference.head(3)

In [ ]:
%cd /content/gdrive/My Drive/Colab/intrapolitical_suicidality/code

from predict import load_model, predict

In [ ]:
models_path = '/content/gdrive/My Drive/Colab/intrapolitical_suicidality/outputs/models/'
pretrained_model_name = 'bert-base-uncased'
model_class = BertForSequenceClassification
tokenizer_class = BertTokenizer
model_path = f'{models_path}pb_bert/pb_best_model.bin'

target = 'pb'

tokenizer = tokenizer_class.from_pretrained(pretrained_model_name)

model = load_model(
                   model_path,
                   model_class,
                   pretrained_model_name,
                   )

#texts = d_reflexive['text'].tolist()
texts = d_inference['text'].tolist()

predictions, probabilities = predict(
                                     model,
                                     tokenizer,
                                     texts,
                                     )

#d_reflexive[f'{target}_label'] = predictions
#d_reflexive[f'{target}_proba'] = probabilities
d_inference[f'{target}_label'] = predictions
d_inference[f'{target}_proba'] = probabilities

#d_reflexive.head(3)
d_inference.head(3)

In [ ]:
%cd /content/gdrive/My Drive/Colab/intrapolitical_suicidality/inputs/data

#d_reflexive.to_excel('d_reflexive_labeled.xlsx')
d_inference.to_csv('d_inference_labeled.csv')

### 6. Causal inference
Transforms $\mathcal{D}$<sub>inference</sub> for causal inference modeling. Collapses analytic sample to _block_ units, concatenates text covariate _$W_{p}$_, computes aggregate sentiment outcome _$Y_{p}$_. Fits intra-textual causal inference Models A and B.
***

In [ ]:
%pwd

#### Autocode: comment sentiment

In [ ]:
d = pd.read_csv('d_inference_labeled.csv')

d.set_index(
             'index',
             inplace = True,
             )

d.shape
d[[
   'hate_label',
   'lone_label',
   'tb_label',
   'pb_label',
   ]].apply(pd.Series.value_counts)

d.head(3)

**Count: _n_ unique posts**

In [ ]:
d['p_uniq'] = (d['text'] != '<|RPT|>').astype(int)
d['p_uniq'].value_counts()

**Disaggregate: post author | commenter comments**

In [ ]:
# Dupe c_text col

d['p_au_c_text'] = d['c_text']

# Dupe c_text text

d.loc[d['p_au_reply'] == 1, 'p_au_c_text'] = d.loc[d['p_au_reply'] == 1, 'c_text']

# Disaggregate about p_author_reply = 1/0

d.loc[d['p_au_reply'] == 1, 'c_text'] = ' '
d.loc[d['p_au_reply'] != 1, 'p_au_c_text'] = ' '

d['p_uniq'].value_counts()
d.head(30)

In [ ]:
# Force to str prior to autocoding

d['c_text'] = d['c_text'].astype(str)
d['p_au_c_text'] = d['p_au_c_text'].astype(str)

# Reduce

d = d[[
       'p_au',
       'p_date',
       'id',
       'n_cmnt',
       'text',
       'p_sbrt',
       'p_titl',
       'c_au',
       'c_sbrt',
       'c_text',
       'new_block',
       'block',
       'p_au_reply',
       'hate_label',
       'lone_label',
       'tb_label',
       'pb_label',
       'p_uniq',
       'p_au_c_text'
       ]].copy()

d.dtypes
d.head(3)

**Autocode**

In [ ]:
ac = Autocoder()

d = ac.code_sentiment(
                      d['c_text'].values,
                      d,
                      batch_size = 8,
                      binarize = False,
                      #threshold = 0.5,
                      )

d.head(30)
d.to_csv('d_inference_labeled_autocoded.csv')

#### Compute outcomes, concatenate text covariates

In [ ]:
# import

d = pd.read_csv('d_inference_labeled_autocoded.csv')

d.rename(
         columns = {'Unnamed: 0': 'index'},
         inplace = True,
         )

d.set_index(
             'index',
             inplace = True,
             )

d.head(3)

**Text covariate _$W_{pA}$_: concatenate post author comments + title (by block)**

In [ ]:
d['p_au_c_text'] = d['p_au_c_text'].astype(str)

# Concat post author comments x block

d['block_p_au_c_text'] = d.groupby('block')['p_au_c_text'].transform(lambda i: ' '.join(i))

# Append post title to concatenated post author comments

d['text_covar_a'] = d.apply(lambda row: row['block_p_au_c_text'] + row['p_titl'] if row['p_titl'] != '<|RPT|>' else row['block_p_au_c_text'], axis = 1)

d['p_uniq'].value_counts()
d.head(3)

**Text covariate _$W_{pB}$_: concatenated post author comments (by block)**

In [ ]:
d.rename(
         columns = {'block_p_au_c_text': 'text_covar_b'},
         inplace = True,
         )

d.dtypes
d.head(3)

In [ ]:
'''
# BACKUP BACKUP BACKUP 01

d.to_csv(
         'd_inference_labeled_autocoded_backup01.csv',
         index = 'index',
         )

In [ ]:
# Delete sentiment fr post author replies

d.loc[d['p_au_reply'] == 1, 'positive'] = None
d.shape
d['p_uniq'].value_counts()
d.head(30)

In [ ]:
# Avg positive sentiment x block

d['pos_block_mean'] = d.groupby('block')['positive'].transform('mean')

# Mdn

mdn = d['positive'].median()

# IQR

q1 = d['positive'].quantile(0.25)
q3 = d['positive'].quantile(0.75)

'mdn'
print(mdn)

'Q1'
print(q1)

'Q3'
print(q3)

**Compute _$Y_{p}$_: >Mdn block-level positive sentiment**

In [ ]:
d['y_pos_mdn'] = (d['pos_block_mean'] >= mdn).astype(int)
d['y_pos_mdn'].value_counts()

d.head(3)

In [ ]:
'''
# BACKUP BACKUP BACKUP 02

d.to_csv(
         'd_inference_labeled_autocoded_backup02.csv',
         index = 'index',
         )

In [ ]:

d = pd.read_csv(
           'd_inference_labeled_autocoded_backup02.csv',
           )

d.head(3)

**Collapse: block-level _$W_{pA}$_, _$W_{pB}$_, _$Y_{p}$_ aggregates for model input**

In [ ]:
# Reduce for LGBMClassifier input

d = d[[
       'text',
       'block',
       'p_uniq',
       'hate_label',
       'lone_label',
       'tb_label',
       'pb_label',
       'p_au_reply',
       'text_covar_a',
       'text_covar_b',
       'y_pos_mdn',
       ]].copy()

In [ ]:
# p_au_reply_block: 1 = _block_ includes a post author reply -> retain for subset

d['p_au_reply_block'] = d.groupby('block')['p_au_reply'].transform(lambda i: int(i.max() == 1))
d['p_au_reply_block'].value_counts()

d.head(30)

In [ ]:
d['p_uniq'].value_counts()
d = d[d['p_uniq'] != 0]

d.reset_index(inplace = True)

d.shape
d.head(30)

In [ ]:
'''
# BACKUP BACKUP BACKUP 03

d.to_csv(
         'd_inference_labeled_autocoded_backup03.csv',
         index = 'index',
         )

**Model A: all blocks ($\mathcal{d}$<sub>all blocks</sub>)**

In [ ]:
d_all_blocks = d

**Model B: restrict to posts with post author replies ($\mathcal{d}$<sub>author blocks</sub>)**

In [ ]:
d_au_blocks = d[d['p_au_reply_block'] == 1]

d_au_blocks.reset_index(inplace = True)

d_au_blocks.shape
d_au_blocks.head(30)

In [ ]:
'''
# BACKUP BACKUP SUB

d_sub.to_csv(
         'd_sub_backup.csv',
         index = 'index',
         )

In [ ]:
#%cd /content/gdrive/My Drive/Colab/intrapolitical_suicidality/inputs/data

#d_all_blocks = pd.read_csv(
#                          'd_inference_labeled_autocoded_backup03.csv',
#                          )

d_all_blocks.head(3)

#d_au_blocks = pd.read_csv(
#                           'd_sub_backup.csv',
#                           )

d_au_blocks.head(3)

    ### SJS 8/12: these col heads match 1:1, so ATE loop is stable - but clean these crappy indices up

**8:2 subset for robustness check**

In [ ]:
# Model A

d_all_blocks = d_all_blocks.drop(columns = ['text_covar_b'])

d_all_blocks_subset = d_all_blocks.sample(
                                          frac = 0.8,
                                          random_state = 56,
                                          )



d_all_blocks_subset.rename(
                           columns = {'text_covar_a': 'text_covar_a_subset'},
                           inplace = True,
                           )

d_all_blocks_subset.reset_index(
                                drop = True,
                                inplace = True,
                                )

d_all_blocks.shape
d_all_blocks_subset.shape

# Model B

d_au_blocks = d_au_blocks.drop(columns = ['text_covar_a'])

d_au_blocks_subset = d_au_blocks.sample(
                                        frac = 0.8,
                                        random_state = 56,
                                        )
d_au_blocks_subset.rename(
                          columns = {'text_covar_b': 'text_covar_b_subset'},
                          inplace = True,
                          )

d_au_blocks_subset.reset_index(
                               drop = True,
                               inplace = True,
                               )


d_au_blocks.shape
d_au_blocks_subset.shape

#### Model A: fit and estimate

In [ ]:
treatments = [
              'hate_label',
              'lone_label',
              'tb_label',
              'pb_label',
              ]

ignore_cols_baseline = [
                        'Unnamed: 0',
                        'level_0',
                        'index',
                        'p_au',
                        'p_date',
                        'text',
                        'p_sbrt',
                        'block',
                        'p_uniq',
                        'p_au_reply',
                        #'text_covar_b',
                        'p_au_reply_block',
                        ]

tokens = [
          'deserve',
          'disabled',
          'ugly',
          'disgust',
          'abuse',
          'sex',
          'lonely',
          'alone',
          ]

models = {}

for treatment in treatments:
    model_name = f'cm_{treatment.split("_")[0]}'
    ignore_cols = ignore_cols_baseline + [t for t in treatments if t != treatment]

    print('\n-----------------------------------------------------------------------')
    print(f'Fitting: {model_name}')
    print('-----------------------------------------------------------------------')
    print('\n')

    models[model_name] = CausalInferenceModel(
                                              d_all_blocks, ### primary model
                                              #d_all_blocks_subset, ### 80% sample: robustness check
                                              method = 't-learner',
                                              learner = LGBMClassifier(num_leaves = 500),
                                              treatment_col = treatment,
                                              outcome_col = 'y_pos_mdn',
                                              text_col = 'text_covar_a', ### 'text_covar_a' = post author comments _and_ post titles
                                              #text_col = 'text_covar_a_subset',
                                              ngram_range=(
                                                           1,
                                                           3
                                                           ),
                                              min_df = 0.01,
                                              stop_words = 'english',
                                              ignore_cols = ignore_cols,
                                              verbose = -1,
                                              )

    models[model_name].fit()

    print('\n=======================================================================')
    print(f'Results: ATE for {model_name}')
    print('=======================================================================')
    ate = models[model_name].estimate_ate()
    print(ate)
    print('\n')

    top_features = models[model_name].interpret(
                                                plot = False,
                                                method = 'feature_importance',
                                                )[1][:10]

    print('\n')
    print('=======================================================================')
    for token in tokens:
        print(f'CATE for {model_name}: {token}')
        models[model_name].estimate_ate(d_au_blocks['text_covar_b'].str.contains(token))

    print('\n=======================================================================')
    print(f'Top features for {model_name}:')
    print('=======================================================================')
    print(top_features)
    print('\n')

#### Model B: fit and estimate

In [ ]:
treatments = [
              'hate_label',
              'lone_label',
              'tb_label',
              'pb_label',
              ]

ignore_cols_baseline = [
                        'Unnamed: 0',
                        'level_0',
                        'index',
                        'p_au',
                        'p_date',
                        'text',
                        'p_sbrt',
                        'block',
                        'p_uniq',
                        'p_au_reply',
                        #'text_covar_a',
                        'p_au_reply_block',
                        ]

tokens = [
          'deserve',
          'disabled',
          'ugly',
          'disgust',
          'abuse',
          'sex',
          'lonely',
          'alone',
          ]

models = {}

for treatment in treatments:
    model_name = f'cm_{treatment.split("_")[0]}'
    ignore_cols = ignore_cols_baseline + [t for t in treatments if t != treatment]

    print('\n-----------------------------------------------------------------------')
    print(f'Fitting: {model_name}')
    print('-----------------------------------------------------------------------')
    print('\n')

    models[model_name] = CausalInferenceModel(
                                              d_au_blocks,  ### primary model
                                              #d_au_blocks_subset,  ### 80% sample: robustness check
                                              method = 't-learner',
                                              learner = LGBMClassifier(num_leaves = 500),
                                              treatment_col = treatment,
                                              outcome_col = 'y_pos_mdn',
                                              text_col = 'text_covar_b',  ### 'text_covar_b' = post author comments
                                              #text_col = 'text_covar_b_subset',
                                              ngram_range=(
                                                           1,
                                                           3
                                                           ),
                                              min_df = 0.01,
                                              stop_words = 'english',
                                              ignore_cols = ignore_cols,
                                              verbose = -1,
                                              )

    models[model_name].fit()

    print('\n=======================================================================')
    print(f'Results: ATE for {model_name}')
    print('=======================================================================')
    ate = models[model_name].estimate_ate()
    print(ate)
    print('\n')

    top_features = models[model_name].interpret(
                                                plot = False,
                                                method = 'feature_importance',
                                                )[1][:10]

    print('\n')
    print('=======================================================================')
    for token in tokens:
        print(f'CATE for {model_name}: {token}')
        models[model_name].estimate_ate(d_au_blocks['text_covar_b'].str.contains(token))

    print('\n=======================================================================')
    print(f'Top features for {model_name}:')
    print('=======================================================================')
    print(top_features)
    print('\n')

**Concordances x token: nltk**

In [ ]:
import nltk
from nltk.text import Text

import nltk
nltk.download('punkt')

# Derived qualitatively (deductively)

tokens = [
          'deserve',
          'disabled',
          'ugly',
          'disgust',
          'abuse',
          'sex',
          'lonely',
          'alone',
          ]

# Derived by feature importance (inductively)

#tokens = ['thanks']

# parse by T_p

d_parsed = d_au_blocks[d_au_blocks['pb_label'] == 1]

# transform to nltk text object

text_col = d_parsed['text_covar_b'].dropna().tolist()
joined_text = ' '.join(text_col)
tokenized_text = nltk.word_tokenize(joined_text)
nltk_text = Text(tokenized_text)

# examine token in context

for token in tokens:
    nltk_text.concordance(
                          token,
                          lines = 50,
                          width = 100,
                          )


> End of its_train_test_pred.ipynb